### About this notebook: 
<font color='grey'>This notebook is used to answer </font><font color='red'>which gene knockout</font> show sensitivity to <font color='red'> certain gene mutation or the mutation of a group of genes.</font> <br/>

<font color='blue'>The functional screening data and omics data for cell lines is from the Depmap and CCLE project from the Broad institute (DepMap Public 20Q3). To use this jupyter notebook and the data which are used in the jupyter notebook, Please cite the following papers</font> <br/>

....our paper

For this DepMap release:
DepMap, Broad (2020): DepMap 20Q3 Public. figshare. Dataset doi:10.6084/m9.figshare.11791698.v2.

For CRISPR datasets:
Robin M. Meyers, Jordan G. Bryan, James M. McFarland, Barbara A. Weir, ... David E. Root, William C. Hahn, Aviad Tsherniak. Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nature Genetics 2017 October 49:1779–1784. doi:10.1038/ng.3984

Dempster, J. M., Rossen, J., Kazachkova, M., Pan, J., Kugener, G., Root, D. E., & Tsherniak, A. (2019). Extracting Biological Insights from the Project Achilles Genome-Scale CRISPR Screens in Cancer Cell Lines. BioRxiv, 720243.

For omics datasets:
Mahmoud Ghandi, Franklin W. Huang, Judit Jané-Valbuena, Gregory V. Kryukov, ... Todd R. Golub, Levi A. Garraway & William R. Sellers. 2019. Next-generation characterization of the Cancer Cell Line Encyclopedia. Nature 569, 503–508 (2019).


In [2]:
import sys
sys.path.append('../scripts/')
import SL
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats 
import numpy as np
import json
import statsmodels.stats.multitest as multi
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
import plotly
import plotly.express as px

#from google.cloud import bigquery

In [14]:
def Cohen_dist(x,y):

    n1 = len(x)
    n2 = len(y)
    s = np.sqrt(((n1 - 1)*(np.std(x))*(np.std(x)) + (n2 - 1) * (np.std(y)) * (np.std(y))) / (n1 + n2 -2))
    d = (np.mean(x) - np.mean(y)) / s
    return(d)
 

### Define the input directory and the output directory: 
<font color='grey'> 
    "input_dir": directory of input data<br/>
    "output_dir":directory of output data<br/>
</font>
This section can be replaced with the Bigquery statements

In [3]:
directories = {"input_dir":"~/Documents/ISB/KG",
               "output_dir":"../Output_SL/"}

input_data = {
         "input_mut": "Depmap_Crispr_data/20Q3/CCLE_mutations.csv",
         "input_depmap": "Depmap_Crispr_data/20Q3/Achilles_gene_effect.csv",
         "input_sample_info": "Depmap_Crispr_data/20Q3/sample_info_Depmap.csv",
        }

### Read tables and convert to matrix

In [4]:

Mut_mat = pd.read_csv(directories['input_dir']+'/'+input_data['input_mut'], sep = '\t') 
Depmap_matrix = pd.read_csv(directories['input_dir']+'/'+input_data['input_depmap'],sep = ',' , index_col = "DepMap_ID") 
sample_info = pd.read_csv(directories['input_dir']+'/'+input_data['input_sample_info'])



/Users/gloria/.pyenv/versions/3.7.7/envs/3.7.7-incov/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Overview of the datasets
The number of samples with mutation data is 1741;
The number of samples with cripr data is 789;
The number of samples with both cripr data and mutation data is 787;

#### rename the gene names

In [5]:
gene_names_old = list(Depmap_matrix.columns.values)
gene_names_new = []
for item in gene_names_old:
    name = item.split(' (')[0]
    gene_names_new.append(name)
Depmap_matrix.columns = gene_names_new

### Selection of variants

In [6]:
print(list(set(Mut_mat['Variant_Classification'])))
selected_variants = ['Splice_Site',
                     'Frame_Shift_Del',
                     'Frame_Shift_Ins',
                     'Nonstop_Mutation',
                     'In_Frame_Del',
                     'In_Frame_Ins',
                     'Missense_Mutation',
                     'Nonsense_Mutation',
                     'Nonstop_Mutation',
                     'Start_Codon_Del',
                     'Start_Codon_Ins',
                     'Start_Codon_SNP',
                     'Stop_Codon_Del',
                     'Stop_Codon_Del',
                     'Stop_Codon_Ins',
                     'De_novo_Start_OutOfFrame']

[nan, 'Frame_Shift_Del', 'Start_Codon_Ins', 'IGR', 'In_Frame_Ins', 'In_Frame_Del', 'Stop_Codon_Del', 'Missense_Mutation', 'Silent', 'De_novo_Start_OutOfFrame', 'Start_Codon_SNP', "5'UTR", 'Intron', "3'UTR", 'Nonsense_Mutation', 'Frame_Shift_Ins', "5'Flank", 'Splice_Site', 'Stop_Codon_Ins', 'Start_Codon_Del', 'Nonstop_Mutation']


#### Selection of Cancerous cells

In [7]:
pancancer_cls = (sample_info.loc[~sample_info['primary_disease'].isin(['Non-Cancerous','Unknown','Engineered','Immortalized'])])
pancancer_cls = pancancer_cls.loc[~(pancancer_cls['primary_disease'].isna())]


###  <font color='red'>Selection of tumor types.
</font>



In [8]:
TCGA_list = [x for x in list(set(pancancer_cls['TCGA_subtype'])) if x == x]

w = widgets.SelectMultiple(
    options=['pancancer'] + TCGA_list  ,
    value=[],
    description='Tumor type',
    disabled=False
)
display(w)

SelectMultiple(description='Tumor type', options=('pancancer', 'GBM', 'PRAD', 'KIRC', 'ALL', 'OV', 'LIHC', 'SC…

In [9]:
print("TCGA tumor types selected are: ")
print(list(w.value))
tumor_type = list(w.value)
#print("tumor types selected are: " + list(w.value))

if tumor_type == ['pancancer']:
    cl_sele = list(pancancer_cls['DepMap_ID'].values)

else:
    tumor_selected = tumor_type
    cl_sele = sample_info.loc[sample_info['TCGA_subtype'].isin((tumor_selected))]['DepMap_ID']  
    cl_sele = list(set(list(Depmap_matrix.index.values)).intersection(set(cl_sele)))
print("No. of cell lines selected in the sample_info file is : " + str(len(cl_sele)))

Samples_with_mut_kd = set(Mut_mat['DepMap_ID']).intersection(cl_sele).intersection(set(Depmap_matrix.index.values))
print("No. of cell lines selected is with both mutation and knockdown data is : " + str(len(Samples_with_mut_kd)))

TCGA tumor types selected are: 
['pancancer']
No. of cell lines selected in the sample_info file is : 1740
No. of cell lines selected is with both mutation and knockdown data is : 787


In [10]:
Mut_mat_sele1 = Mut_mat.loc[Mut_mat['DepMap_ID'].isin(Samples_with_mut_kd)]
Mut_mat_sele2 = Mut_mat_sele1.loc[Mut_mat_sele1['Variant_Classification'].isin(selected_variants)]

#### Input mutated genes

In [11]:
mut_gene = pd.read_csv("../Github_version/SyntheticLethality/data/CancerDriverGenes.csv")
mut_gene = list(mut_gene['HGNC_gene_symbol'])


In [12]:
Mut_mat_sele3 = Mut_mat_sele2.loc[Mut_mat_sele2['Hugo_Symbol'].isin(mut_gene),['Hugo_Symbol','DepMap_ID']]
Depmap_matrix_sele = Depmap_matrix.loc[Samples_with_mut_kd,:].transpose()

#### T-test and effect size estimation between the mutated group and the wt-group

In [15]:
Gene_mut_list = []
Gene_kd_list = []
p_list = []
es_list = []
size_mut = []
FDR_List = []
for Gene in mut_gene:
    print(Gene)
    p_list_curr = []
    
    Mut_group = list(Mut_mat_sele3.loc[Mut_mat_sele3['Hugo_Symbol'] == Gene]['DepMap_ID'].values)
    WT_group = list(set(Samples_with_mut_kd) - set(Mut_group))

    for Gene_kd in list(Depmap_matrix_sele.index.values):
        D_mut_new = Depmap_matrix_sele.loc[Gene_kd,Mut_group].values
        D_wt_new = Depmap_matrix_sele.loc[Gene_kd,WT_group].values
        
        nan_array = np.isnan(D_mut_new)
        not_nan_array = ~ nan_array
        D_mut_new = D_mut_new[not_nan_array]
        
        nan_array = np.isnan(D_wt_new)
        not_nan_array = ~ nan_array
        D_wt_new = D_wt_new[not_nan_array]
        
        
        if len(D_mut_new) > 2:
            size_mut.append(len(D_mut_new))
            Sci_test = stats.ttest_ind(D_mut_new, D_wt_new, nan_policy = 'omit')
            pvalue = Sci_test[1]
            p_list_curr.append(pvalue)
            
            Size_effect =Cohen_dist(D_mut_new, D_wt_new)
            es_list.append(Size_effect)
            Gene_mut_list.append(Gene)
            Gene_kd_list.append(Gene_kd)
    
    if len(p_list_curr) > 0:
        p_list = p_list + p_list_curr
        FDR_List_table = multi.multipletests(p_list_curr, alpha=0.05, method='fdr_bh', is_sorted=False)[1]
        FDR_List = FDR_List + list(FDR_List_table)


ABL1
ACVR1
ACVR1B
ACVR2A
AJUBA
AKT1
ALB
ALK
AMER1
APC
APOB
AR
ARAF
ARHGAP35
ARID1A
ARID2
ARID5B
ASXL1
ASXL2
ATF7IP
ATM
ATR
ATRX
ATXN3
AXIN1
AXIN2
B2M
BAP1
BCL2
BCL2L11
BCOR
BRAF
BRCA1
BRCA2
BRD7
BTG2
CACNA1A
CARD11
CASP8
CBFB
CBWD3
CCND1
CD70
CD79B
CDH1
CDK12
CDK4
CDKN1A
CDKN1B
CDKN2A
CDKN2C
CEBPA
CHD3
CHD4
CHD8
CHEK2
CIC
CNBD1
COL5A1
CREB3L3
CREBBP
CSDE1
CTCF
CTNNB1
CTNND1
CUL1
CUL3
CYLD
CYSLTR2
DACH1
DAZAP1
DDX3X
DHX9
DIAPH2
DICER1
DMD
DNMT3A
EEF1A1
EEF2
EGFR
EGR3
EIF1AX
ELF3
EP300
EPAS1
EPHA2
EPHA3
ERBB2
ERBB3
ERBB4
ERCC2
ESR1
EZH2
FAM46D
FAT1
FBXW7
FGFR1
FGFR2
FGFR3
FLNA
FLT3
FOXA1
FOXA2
FOXQ1
FUBP1
GABRA6
GATA3
GNA11
GNA13
GNAQ
GNAS
GPS2
GRIN2D
GTF2I
H3F3A
H3F3C
HGF
HIST1H1C
HIST1H1E
HLA-A
HLA-B
HRAS
HUWE1
IDH1
IDH2
IL6ST
IL7R
INPPL1
IRF2
IRF6
JAK1
JAK2
JAK3
KANSL1
KDM5C
KDM6A
KEAP1
KEL
KIF1A
KIT
KLF5
KMT2A
KMT2B
KMT2C
KMT2D
KRAS
KRT222
LATS1
LATS2
LEMD2
LZTR1
MACF1
MAP2K1
MAP2K4
MAP3K1
MAP3K4
MAPK1
MAX
MECOM
MED12
MEN1
MET
MGA
MGMT
MLH1
MSH2
MSH3
MSH6
MTOR
MUC6
MYC
MYCN
MYD88
MYH

In [18]:
result = pd.DataFrame({"Gene_mut": Gene_mut_list, 
                       "Gene_kd": Gene_kd_list, 
                       "Mutated_samples":size_mut,
                       "pvalue": p_list, 
                       "ES":es_list, 
                       "FDR": FDR_List
                      })

In [19]:
result.sort_values(by = ['FDR']).to_csv("../Results/driver_gene_mutation_dependency_crispr"+'.'.join(tumor_type)+"Mar24.2021.csv")